In [52]:
import os
import re
from tqdm import tqdm
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import torch
import string
%matplotlib inline

In [53]:
if torch.cuda.is_available(): 
    device = torch.device("cuda")
    print(f'There are {torch.cuda.device_count()} GPU(s) available.')
    print('Device name:', torch.cuda.get_device_name(0))
else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

There are 2 GPU(s) available.
Device name: GeForce RTX 2080 SUPER


In [54]:
id_list = ['018_087_123','022_094','041_050_095','042_072_169','058_093_144']
filename = '058_093_144'
df_sentiment = pd.read_csv('../Data/Junior_Data/' + filename + '.csv')
df_sentiment

,News_Title,News_Intro,News_Desc,News_All,Gambling,Murder,Sexual Abuse,Theft/Burglary,Drug,Battery/Assault,Accident,Non-Crime
0,ร่วมทำบุญครบ7วัน อุทิศ'แหม่มเยอรมัน'เหยื่อข่มข...,"ชาวเกาะสีชัง ร่วมทำบุญอุทิศส่วนกุศลให้ ""แหม่มเ...",จากกรณีตำรวจจับกุม นายรณกร ร่มรื่น อายุ 23 ปี ...,ร่วมทำบุญครบ7วัน อุทิศ'แหม่มเยอรมัน'เหยื่อข่มข...,0,1,0,0,0,0,0,0
1,กอดคอขอเคลียร์ปัญหาอริเก่า ไม่ลงตัวถูกกดหัวจ่อ...,เพื่อนร่วมห้องเรียนกลายเป็นอริ นัดกอดคอขอเคลีย...,เมื่อเวลา 15.30 น. วันที่ 15 เม.ย. ร.ต.อ.สุระ ...,กอดคอขอเคลียร์ปัญหาอริเก่า ไม่ลงตัวถูกกดหัวจ่อ...,0,1,0,0,0,0,0,0
2,แจ้งจับ'ทอม'แสบลวงสาว18 เข้า'ม่านรูด'ก่อนข่มขื...,พ่อพาลูกสาววัย 18 ปี เข้าแจ้งความเอาผิด สาวทอม...,จากกรณีพ่อของหญิงสาวรายหนึ่ง ได้ร้องเรียนผ่านเ...,แจ้งจับ'ทอม'แสบลวงสาว18 เข้า'ม่านรูด'ก่อนข่มขื...,0,0,1,0,0,0,0,0
3,จับแล้วมือยิงอริเพื่อนร่วมชั้นดับ รับสุมแค้นนา...,แก้แค้น10ปียังไม่สาย!! รวบมือยิงหัวอริอดีตเพื่...,จากกรณีคนร้ายจ่อยิงศีรษะ นายสราวุธ ลาวงศ์ อายุ...,จับแล้วมือยิงอริเพื่อนร่วมชั้นดับ รับสุมแค้นนา...,0,1,0,0,0,0,0,0
4,อย่าอ้างสงกรานต์ทำผิดได้ คึกมือล้วงต่ำลวนลามสาว,สัปดาห์นี้ฝากแง่คิดหลังจบ “เทศกาลสงกรานต์” อย่...,“งัดถั่งงัดถั่งงัดถั่งงัด งัดถั่งงัดถั่งงัดถั่...,อย่าอ้างสงกรานต์ทำผิดได้ คึกมือล้วงต่ำลวนลามสา...,0,0,1,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...
1036,ตร.แจง ดารากุเรื่องเป็นเซียนหวย อาจเข้าข่ายผิด...,รองโฆษก ตร.แจงกรณีสื่อแฉดาราสร้างเรื่องหลอกลวง...,รองโฆษก ตร.แจงกรณีสื่อแฉดาราสร้างเรื่องหลอกลวง...,ตร.แจง ดารากุเรื่องเป็นเซียนหวย อาจเข้าข่ายผิด...,0,0,0,0,0,0,0,1
1037,หนุ่มชาวบ้านที่ถูกยิงดับใต้สะพานที่บาเจาะ ตร.ค...,พบศพหนุ่มวัย 22 ปี ถูกยิงด้วยปืน 9 มม. ตาย 1 ศ...,พบศพหนุ่มวัย 22 ปี ถูกยิงด้วยปืน 9 มม. ตาย 1 ศ...,หนุ่มชาวบ้านที่ถูกยิงดับใต้สะพานที่บาเจาะ ตร.ค...,0,1,0,0,0,0,0,0
1038,ผกก.สายไหม มอบสิ่งของผู้สูงอายุ เนื่องในวันเฉล...,ผกก.สน.สายไหม นำทีมข้าราชการตำรวจในสังกัด และเ...,ผกก.สน.สายไหม นำทีมข้าราชการตำรวจในสังกัด และเ...,ผกก.สายไหม มอบสิ่งของผู้สูงอายุ เนื่องในวันเฉล...,0,0,0,0,0,0,0,1
1039,ในหลวงพระราชทานตราสัญลักษณ์ประจำหน่วยปฏิบัติกา...,พล.ต.ต.จิรภพ ภูริเดช ผบก.ป. พร้อมข้าราชการตำรว...,พล.ต.ต.จิรภพ ภูริเดช ผบก.ป. พร้อมข้าราชการตำรว...,ในหลวงพระราชทานตราสัญลักษณ์ประจำหน่วยปฏิบัติกา...,0,0,0,0,0,0,0,1


In [55]:
X = df_sentiment.iloc[:,3:4].values
y = df_sentiment.iloc[:,4:].values


In [56]:
# !pip install sentencepiece
from transformers import AutoTokenizer, AutoModelForMaskedLM, CamembertTokenizer
# cstorm125/wangchanberta-base-att-spm-uncased-finetune
# airesearch/wangchanberta-base-att-spm-uncased
tokenizer = CamembertTokenizer.from_pretrained("airesearch/wangchanberta-base-att-spm-uncased")

def preprocessing_for_bert(data):

    input_ids = []
    attention_masks = []

    for sent in data:
        encoded_sent = tokenizer.encode_plus(
            #text=text_preprocessing(sent),  
            text=sent,
            add_special_tokens=True,        
            max_length=MAX_LEN,
            truncation=True,             
            padding='max_length',         
            #return_tensors='pt',           
            return_attention_mask=True      
        )
        
        input_ids.append(encoded_sent.get('input_ids'))
        attention_masks.append(encoded_sent.get('attention_mask'))

    input_ids = torch.tensor(input_ids)
    attention_masks = torch.tensor(attention_masks)

    return input_ids, attention_masks

In [57]:
X_train_new = np.array([text for sub in X for text in sub]) 

In [58]:
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
MAX_LEN = 500
batch_size = 10
test_inputs, test_masks = preprocessing_for_bert(X_train_new)
test_dataset = TensorDataset(test_inputs, test_masks)
test_sampler = SequentialSampler(test_dataset)
test_dataloader = DataLoader(test_dataset, sampler=test_sampler, batch_size=batch_size)

In [59]:
import torch.nn.functional as F
from transformers import CamembertForSequenceClassification , BertForSequenceClassification
import torch.nn as nn
import torch
bert_classifier_load = CamembertForSequenceClassification.from_pretrained('airesearch/wangchanberta-base-att-spm-uncased', num_labels=8, output_attentions=False, output_hidden_states=False)
bert_classifier_load = nn.DataParallel(bert_classifier_load)
bert_classifier_load.load_state_dict(torch.load('./Selected_Model/WangChanBERT_epoch100.h5'))
bert_classifier_load.to(device)
bert_classifier_load.eval()

def bert_predict(model, test_dataloader):
    """Perform a forward pass on the trained BERT model to predict probabilities
    on the test set.
    """
    # Put the model into the evaluation mode. The dropout layers are disabled during
    # the test time.
    model.eval()

    all_logits = []

    # For each batch in our test set...
    for batch in test_dataloader:
        # Load batch to GPU
        b_input_ids, b_attn_mask = tuple(t.to(device) for t in batch)[:2]

        # Compute logits
        with torch.no_grad():
            outputs = model(b_input_ids, b_attn_mask)
            logits = outputs.logits
        all_logits.append(logits)
    
    # Concatenate logits from each batch
    all_logits = torch.cat(all_logits, dim=0)

    # Apply softmax to calculate probabilities
    probs = all_logits.sigmoid().cpu().numpy()

    return probs

Some weights of the model checkpoint at airesearch/wangchanberta-base-att-spm-uncased were not used when initializing CamembertForSequenceClassification: ['lm_head.layer_norm.weight', 'lm_head.decoder.weight', 'roberta.pooler.dense.weight', 'lm_head.bias', 'roberta.pooler.dense.bias', 'lm_head.dense.weight', 'lm_head.layer_norm.bias', 'lm_head.dense.bias', 'lm_head.decoder.bias']
- This IS expected if you are initializing CamembertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing CamembertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of CamembertForSequenceClassification were not initialized from the model checkpoint at airesearch/wa

In [60]:
probs = bert_predict(bert_classifier_load, test_dataloader)

In [61]:
print(probs[0])

[5.7272620e-05 9.9994433e-01 9.9998105e-01 2.3638023e-04 8.0907793e-04
 2.2450901e-04 4.1838386e-05 3.5913286e-05]


In [62]:
print(y[0])

[0 1 0 0 0 0 0 0]


In [63]:
from sklearn.metrics import classification_report

# start from 5 since we need to avoid the news_data <Topic,Intro,Desc,All>
df_label_columns = df_sentiment.columns[4:]
label_names = list(df_label_columns)
print(label_names)

print(classification_report(y, np.round(probs), target_names=label_names, zero_division=0))
df_label_columns

['Gambling', 'Murder', 'Sexual Abuse', 'Theft/Burglary', 'Drug', 'Battery/Assault', 'Accident', 'Non-Crime']
                 precision    recall  f1-score   support

       Gambling       0.47      0.69      0.56        13
         Murder       0.82      0.88      0.85       352
   Sexual Abuse       0.81      0.89      0.84        89
 Theft/Burglary       0.60      0.91      0.72        89
           Drug       0.77      0.85      0.80       162
Battery/Assault       0.38      0.88      0.53       100
       Accident       0.34      0.70      0.46        40
      Non-Crime       0.72      0.52      0.60       196

      micro avg       0.66      0.80      0.72      1041
      macro avg       0.61      0.79      0.67      1041
   weighted avg       0.71      0.80      0.73      1041
    samples avg       0.70      0.80      0.74      1041



Index(['Gambling', 'Murder', 'Sexual Abuse', 'Theft/Burglary', 'Drug',
       'Battery/Assault', 'Accident', 'Non-Crime'],
      dtype='object')

In [64]:
import pickle

with open('./pickle_files/Prob_Predict' + '_' + filename + '.pickle', 'wb') as prob:
    pickle.dump(probs, prob)